In [2]:
import numpy as np
import pandas as pd
df_original = pd.read_csv("D:/Portfolio/eu_sdg_performance_data.csv")

We will provide a description of the variables included in the dataset in accordance with the methodology of the Europe Sustainable Development Report 2025 (ESDR2025).

| Variable              | Description                                                    | Unit in dataset                                 |
| --------------------- | -------------------------------------------------------------- | ----------------------------------------------- |
| **year**              | Year of observation                                            | year (2015–2024)                                |
| **sdgi\_score**       | Overall SDG Index (aggregate sustainability performance score) | scale 0–100                                     |
| **performance\_tier** | Sustainability performance classification                      | 0 = Needs Improvement, 1 = Good, 2 = Leaders    |
| **sdg8\_income**      | Average income per capita (GDP per capita, PPP)                | € per person (≈ 8,800 – 39,000)                 |
| **sdg8\_unemp**       | Unemployment rate                                              | % of population (2 – 26%)                       |
| **sdg8\_inwork**      | Employment-to-population ratio                                 | % of population (3 – 19%)                       |
| **sdg9\_eurd**        | Expenditure on R\&D                                            | % of GDP (0 – 4%)                               |
| **sdg9\_digital**     | Basic digital skills of the population                         | % of population (28 – 83%)                      |
| **sdg9\_bband**       | Broadband internet coverage                                    | % of households (59 – 100%)                     |
| **sdg16\_cpi**        | Corruption Perception Index (Transparency International)       | scale 0–100 (34 – 91; higher = less corruption) |
| **sdg16\_rsf**        | Press Freedom Index (Reporters Without Borders)                | scale 0–100 (32 – 95; higher = freer press)     |
| **sdg16\_crime**      | Crime/Safety index                                             | normalized index 1–26 (lower = less crime)      |
| **sdg4\_tertiary**    | Population with tertiary education                             | % of adults (23 – 63%)                          |
| **sdg4\_adult**       | Adult participation in education/training                      | % of adults (1 – 39%)                           |
| **sdg7\_eurenew**     | Share of renewable energy in final energy consumption          | % (5 – 84%)                                     |
| **sdg13\_co2gcp**     | CO₂ emissions per capita                                       | tons CO₂ per person (≈ 3 – 16.6)                |
| **sdg5\_empgap**      | Gender employment gap                                          | % difference between men and women (0 – 43%)    |
| **sdg10\_gini**       | Gini coefficient (income inequality)                           | 21 – 45 (scale typical for Europe)              |


In [3]:
countries = sorted(df_original["Country"].unique())
for c in countries:
    print(repr(c)) 

'Austria'
'Baltic States'
'Belgium'
'Bulgaria'
'Candidate Countries'
'Central and Eastern Europe'
'Croatia'
'Cyprus'
'Czechia'
'Denmark'
'EFTA Countries'
'Estonia'
'European Union'
'Finland'
'France'
'Germany'
'Greece'
'Hungary'
'Iceland'
'Ireland'
'Italy'
'Latvia'
'Lithuania'
'Luxembourg'
'Malta'
'Netherlands'
'North Macedonia'
'Northern Europe'
'Norway'
'Poland'
'Portugal'
'Romania'
'Serbia'
'Slovak Republic'
'Slovenia'
'Southern Europe'
'Spain'
'Sweden'
'Switzerland'
'TŸrkiye'
'United Kingdom'
'Western Europe'


In the original dataset, the Country column contained a mix of individual nations (e.g., Germany, Poland, Romania) and regional aggregations (European Union, Baltic States, Candidate Countries). 

Potential problems:
1. Inconsistent Granularity: the dataset contains information at two different levels of detail: individual countries (low aggregation) and regional groups (high aggregation). This inconsistency can distort the analysis, as the model would be trained on data with varying levels of granularity.
2. Alignment with Project Goal: the main project goal is to classify individual European countries. Including aggregated data that describes regional groups contradicts this objective. Removing this data ensures the model focuses exclusively on the features relevant to classifying individual nations.
3. Bias: aggregated data represents average indicators for entire regions and does not reflect the unique characteristics of any single country. Using it can introduce bias, as the model would be trained on "hybrid" data points that are not representative of the real classification subjects.

Removing aggregated data simplifies the model and makes it more understandable. This allows us to be confident that the conclusions we draw from the model (for example, identifying key factors influencing the classification) truly relate to individual countries and not to the averaged characteristics of entire regions. This makes our results more reliable and meaningful for decision-making.

After cleaning, 34 countries remained (approximately 81% of all rows). This provides a sufficient base for modeling and makes the results more accurate and reliable.

In [4]:
regions_to_exclude = [
    'Baltic States',
    'Candidate Countries',
    'Central and Eastern Europe',
    'EFTA Countries',
    'European Union',
    'Northern Europe',
    'Southern Europe',
    'Western Europe'
]
df= df_original[~df_original['Country'].isin(regions_to_exclude)].copy()
df_regions = df_original[df_original['Country'].isin(regions_to_exclude)].copy()

Replace the county's name:

In [5]:
df["Country"] = df["Country"].replace("TŸrkiye", "Turkey")

For the following analysis, we need to properly impute all NaN values.
Let's find the number of NaN values and the countries/columns that contain them.

In [6]:
print(df.info())

nan_per_column = df.isnull().sum()
print("\nNaN per column:\n", nan_per_column)

total_nan = df.isnull().sum().sum()
print("\nTotal number of NaN:", total_nan)

missing_dict = (
    df.drop(columns=["Country"])
      .groupby(df["Country"])
      .apply(lambda g: g.isna().any()[lambda x: x].index.tolist())
      .to_dict()
) 
missing_nonempty = {k: v for k, v in missing_dict.items() if v}
print("\nCountries with missing values (NaN):")
for country, cols in missing_nonempty.items():
    print(f"\t{country}: {cols}")

cols_with_nan = df.columns[df.isna().any()].tolist()
print("\nColumns with missing values (NaN):", cols_with_nan)

<class 'pandas.core.frame.DataFrame'>
Index: 340 entries, 0 to 409
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country           340 non-null    object 
 1   year              340 non-null    int64  
 2   sdgi_score        340 non-null    float64
 3   performance_tier  340 non-null    int64  
 4   sdg8_income       330 non-null    float64
 5   sdg8_unemp        330 non-null    float64
 6   sdg8_inwork       340 non-null    int64  
 7   sdg9_eurd         340 non-null    int64  
 8   sdg9_digital      330 non-null    float64
 9   sdg9_bband        340 non-null    int64  
 10  sdg16_cpi         340 non-null    int64  
 11  sdg16_rsf         340 non-null    int64  
 12  sdg16_crime       340 non-null    int64  
 13  sdg4_tertiary     340 non-null    int64  
 14  sdg4_adult        340 non-null    int64  
 15  sdg7_eurenew      310 non-null    float64
 16  sdg13_co2gcp      340 non-null    float64
 17  sd

Unfortunately, we cannot use real data from other sources (the World Bank or Eurostat), as the research methodologies differ and the required data are simply unavailable.
That's why we will follow the strategy: imputation will be based on the regional mean.

df — the main dataset with individual countries
df_regions — the dataset with aggregated groups (Candidate Countries, EFTA Countries, ...). 

The df_regions subset already contains averaged indicators for the corresponding country groups; therefore, during imputation we will substitute the values for the respective year from the appropriate group:

North Macedonia → Candidate Countries 
Switzerland → EFTA Countries
Turkey → Candidate Countries
United Kingdom → Western Europe

In [7]:
#This dictionary defines the mapping between an individual country and its corresponding aggregated group (union).
union_map = {
    'North Macedonia': 'Candidate Countries',
    'Switzerland': 'EFTA Countries',
    'Turkey': 'Candidate Countries',
    'United Kingdom': 'Western Europe',
}
#  missing_nonempty -  this dictionary specifies which variables are missing for each country

#We will create new function impute_from_unions
def impute_from_unions(df, df_regions, union_map, missing_nonempty,
                       country_col='Country', year_col='year'):
    out = df.copy()
    for country, cols in missing_nonempty.items():
        union = union_map.get(country)
        if not union:
            continue
        for col in cols:
            if col not in out.columns or col not in df_regions.columns:
                continue
            # серія рік -> значення з union/aggregated group
            union_series = (
                df_regions[df_regions[country_col] == union][[year_col, col]]
                .dropna()
                .set_index(year_col)[col]
            )
            if union_series.empty:
                continue
            mask = (out[country_col] == country) & (out[col].isna())
            out.loc[mask, col] = out.loc[mask, year_col].map(union_series)
    return out


df = impute_from_unions(df, df_regions, union_map, missing_nonempty)

#Let's revise the results of data preprocessing
total_nan = df.isnull().sum().sum()
print("\nTotal number of NaN:", total_nan)


 0tal number of NaN:


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from scipy.stats import randint

# 1) Завантаження даних
df = pd.read_csv("cleaned_data.csv")

# 2) X / y
X = df.drop(columns=["performance_tier", "sdgi_score", "Country", "year"], errors="ignore")
y = df["performance_tier"]

# 3) Train / test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 4) Random Forest + RandomizedSearch
pipe_rf = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("rf", RandomForestClassifier(random_state=42, n_jobs=-1))
])

param_dist_rf = {
    "rf__n_estimators": randint(300, 1200),
    "rf__max_depth": randint(3, 50),
    "rf__min_samples_split": randint(2, 20),
    "rf__min_samples_leaf": randint(1, 10),
    "rf__max_features": ["sqrt", "log2", None],
    # "rf__class_weight": [None, "balanced", "balanced_subsample"],  # опційно
}

rs_rf = RandomizedSearchCV(
    pipe_rf, param_distributions=param_dist_rf, n_iter=50,
    cv=cv, scoring="f1_weighted", n_jobs=-1, random_state=42, refit=True
)

# 5) Навчання
rs_rf.fit(X_train, y_train)
y_pred_rf = rs_rf.predict(X_test)

# 6) Результати
print("=== Random Forest (RandomizedSearch) ===")
print("Best params:", rs_rf.best_params_)
print("CV best f1_weighted:", rs_rf.best_score_)
print("Test Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Test F1 (weighted):", f1_score(y_test, y_pred_rf, average="weighted"))
print("\nConfusion matrix:\n", confusion_matrix(y_test, y_pred_rf))
print("\nClassification report:\n", classification_report(y_test, y_pred_rf))

# 7) Топ-важливості ознак
final_rf = rs_rf.best_estimator_.named_steps["rf"]
importances = pd.Series(final_rf.feature_importances_, index=X.columns).sort_values(ascending=False)
print("\nTop-10 RF feature importances:")
print(importances.head(10))

=== Random Forest (RandomizedSearch) ===
Best params: {'rf__max_depth': 11, 'rf__max_features': None, 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 5, 'rf__n_estimators': 900}
CV best f1_weighted: 0.9418263191846211
Test Accuracy: 0.9607843137254902
Test F1 (weighted): 0.9600802729891631

Confusion matrix:
 [[18  3  0]
 [ 0 65  0]
 [ 0  1 15]]

Classification report:
               precision    recall  f1-score   support

           0       1.00      0.86      0.92        21
           1       0.94      1.00      0.97        65
           2       1.00      0.94      0.97        16

    accuracy                           0.96       102
   macro avg       0.98      0.93      0.95       102
weighted avg       0.96      0.96      0.96       102


Top-10 RF feature importances:
sdg9_digital     0.382087
sdg16_cpi        0.197545
sdg9_eurd        0.088927
sdg4_tertiary    0.064170
sdg7_eurenew     0.055871
sdg16_rsf        0.048062
sdg13_co2gcp     0.039375
sdg10_gini       0.039187
sd